# Human evaluation script for story comparison

In [ ]:
import json
import os.path
from PIL import Image
from matplotlib import pyplot as plt
from numpy import random
from collections import Counter
import sys

sys.path.append('../')
from rule_mining import utils as ut
from rule_mining import config as cfg

XE_pred_file = "/home/lijiacheng/AREL-master/data/save/XE/XE_b3.json"
seman_XE_file = "/home/lijiacheng/VSCMR/data/save/XE2_2/XE2-2_b1.json"
eval_list_file = "eval_list.txt"
image_dir = os.path.join(cfg.dataset_path, 'images/test')

XE_stories = ut.load_json(XE_pred_file)['output_stories']
seman_XE_stories = ut.load_json(seman_XE_file)['output_stories']

eval_list = []
with open(eval_list_file, "r") as f:
    for line in f:
        eval_list.append(int(line))

In [ ]:
def convert_to_dict(stories):
    story_dict = {}
    for story in stories:
        story_dict[int(story['story_id'])] = story
    stories = story_dict
    return stories
XE_stories = convert_to_dict(XE_stories)
seman_XE_stories = convert_to_dict(seman_XE_stories)
XE_stories_keys = [int(idx) for idx in XE_stories.keys()]
seman_XE_stories_keys = [int(idx) for idx in seman_XE_stories.keys()]

def judge(human_label, seman_label):
    if human_label == "C" or human_label == 'c': 
        score = 0
    elif human_label == seman_label or human_label == seman_label.lower():
        score = 1
    else:
        score = -1
    return score

def count(score_list):
    result = {}
    counter = Counter()
    for tpl in score_list:
        counter.update([tpl[0]])
    ssum = counter[-1] + counter[0] + counter[1]
    result['win'] = counter[1] / float(ssum)
    result['lose'] = counter[-1] / float(ssum)
    result['tie'] = counter[0] / float(ssum)
    return result

In [ ]:
tester_name = raw_input("Please input your name:")
relevance, expressiveness, correctness = [], [], []
for idx,story_id in enumerate(eval_list[:1]):
    if story_id in XE_stories_keys and story_id in seman_XE_stories_keys:
        plt.figure(figsize=(17, 90))
        for i in range(5):
            base_name = XE_stories[story_id]['photo_sequence'][i] + ".jpg"
            filename = os.path.join(image_dir, base_name)
            image = Image.open(filename)
            plt.subplot(1, 5, 1+i)
            plt.imshow(image)
            plt.axis('off') 
        plt.show()
        print "NO.",idx,"story_id:",story_id,'\n'
        order = random.randint(0, 2)
        if order == 1:
            print "A:",XE_stories[story_id]['story_text_normalized'],'\n'
            print "B:",seman_XE_stories[story_id]['story_text_normalized'],'\n'
            scorer = judge(raw_input("Which story is more related to images:"), 'B')
            relevance.append((scorer, story_id))
            scoree = judge(raw_input("Which story is more expressive:"), 'B')
            expressiveness.append((scoree, story_id))
            scorec = judge(raw_input("Which story is more concrete:"), 'B')
            correctness.append((scorec, story_id))
            #print scorer,scoree,scorec
        else:
            print "A:",seman_XE_stories[story_id]['story_text_normalized'],'\n'
            print "B:",XE_stories[story_id]['story_text_normalized'],'\n'
            scorer = judge(raw_input("Which story is more related to images:"), 'A')
            relevance.append((scorer, story_id))
            scoree = judge(raw_input("Which story is more expressive:"), 'A')
            expressiveness.append((scoree, story_id))
            scorec = judge(raw_input("Which story is more concrete:"), 'A')
            correctness.append((scorec, story_id))
            #print scorer,scoree,scorec
            
test_result = {"relevance":count(relevance),
              "expressiveness":count(expressiveness),
              "correctness":count(correctness)}
test_record = {"relevance":relevance,
              "expressiveness":expressiveness,
              "correctness":correctness,
              "result":test_result}            

print(test_result)
save_dir = os.path.join(cfg.proj_dir, 'data/eval/')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
ut.save_json(os.path.join(save_dir, tester_name +'.json'), test_record)